In [1]:
#!pip install matplotlib
#!pip install tensorflow


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


#  Preparing Data

In [1]:
import os
# Directory to check for existence
train_dir = '../train'
# Directory to check for existence
valid_dir = '../valid'



In [2]:
height,width=180,180



In [19]:
## Augment the data
#data_augmentation = tf.keras.Sequential([
#    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
#    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
#    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
#    tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
#    tf.keras.layers.experimental.preprocessing.RandomTranslation(
#        height_factor=0.2, 
#        width_factor=0.2, 
#        fill_mode='reflect'
#    ),
#])


In [10]:
#train_set = tf.keras.preprocessing.image_dataset_from_directory(train_dir,image_size=(height,width))
#valid_set = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,image_size=(height,width))



train_set = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(height, width),
    shuffle=True,
)
class_names = train_set.class_names
#train_set = train_set.map(lambda x, y: (data_augmentation(x, training=True), y))

valid_set = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    image_size=(height, width),
)

batches = 10
train_set = train_set.take(batches)
valid_set = valid_set.take(batches)


Found 81946 files belonging to 10 classes.
Found 10244 files belonging to 10 classes.


In [6]:
train_set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:
valid_set = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,image_size=(height,width))

Found 10244 files belonging to 10 classes.


In [8]:
valid_set

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
from tensorflow.python.keras.layers import Dense, Flatten


## Create sequential model and download weights

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

# Define your model architecture
model = Sequential([
    # Add layers according to your requirement
    Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # Adjust the number of units to match the number of classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_set, validation_data=valid_set, epochs=10)
model.save_weights('sequential.h5')



Epoch 1/10
20/20 [==============================] - 25s 1s/step - loss: 9590.3555 - accuracy: 0.1156 - val_loss: 1518.0831 - val_accuracy: 0.2259
Epoch 2/10
20/20 [==============================] - 25s 1s/step - loss: 443.2111 - accuracy: 0.3703 - val_loss: 57.7588 - val_accuracy: 0.3708
Epoch 3/10
20/20 [==============================] - 27s 1s/step - loss: 13.9070 - accuracy: 0.7250 - val_loss: 37.3726 - val_accuracy: 0.3914
Epoch 4/10
20/20 [==============================] - 25s 1s/step - loss: 5.3668 - accuracy: 0.8531 - val_loss: 25.9697 - val_accuracy: 0.4610
Epoch 5/10
20/20 [==============================] - 25s 1s/step - loss: 1.9982 - accuracy: 0.9281 - val_loss: 25.1817 - val_accuracy: 0.4737
Epoch 6/10
20/20 [==============================] - 24s 1s/step - loss: 1.0510 - accuracy: 0.9609 - val_loss: 25.7674 - val_accuracy: 0.4844
Epoch 7/10
20/20 [==============================] - 24s 1s/step - loss: 1.6833 - accuracy: 0.9594 - val_loss: 26.5197 - val_accuracy: 0.4705
Epoch

## Create ResNet50 model and download weights

In [11]:
dnn_model = Sequential()

imported_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180, 180, 3),
    pooling='avg',
    classes=10,
    weights='imagenet')

# Freeze all layers in the imported model
for layer in imported_model.layers:
    layer.trainable = False

# Check if the layers are trainable
for layer in imported_model.layers:
    print(f"Layer {layer.name}: Trainable = {layer.trainable}")


dnn_model.add(imported_model)
dnn_model.add(Flatten())
#dnn_model.add(Dense(600, activation='relu'))
#could go back to 600 later
dnn_model.add(Dense(256, activation='relu'))
dnn_model.add(Dense(10, activation='softmax'))



#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
dnn_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])


Layer input_1: Trainable = False
Layer conv1_pad: Trainable = False
Layer conv1_conv: Trainable = False
Layer conv1_bn: Trainable = False
Layer conv1_relu: Trainable = False
Layer pool1_pad: Trainable = False
Layer pool1_pool: Trainable = False
Layer conv2_block1_1_conv: Trainable = False
Layer conv2_block1_1_bn: Trainable = False
Layer conv2_block1_1_relu: Trainable = False
Layer conv2_block1_2_conv: Trainable = False
Layer conv2_block1_2_bn: Trainable = False
Layer conv2_block1_2_relu: Trainable = False
Layer conv2_block1_0_conv: Trainable = False
Layer conv2_block1_3_conv: Trainable = False
Layer conv2_block1_0_bn: Trainable = False
Layer conv2_block1_3_bn: Trainable = False
Layer conv2_block1_add: Trainable = False
Layer conv2_block1_out: Trainable = False
Layer conv2_block2_1_conv: Trainable = False
Layer conv2_block2_1_bn: Trainable = False
Layer conv2_block2_1_relu: Trainable = False
Layer conv2_block2_2_conv: Trainable = False
Layer conv2_block2_2_bn: Trainable = False
Layer co

/Users/EdenChung/anaconda3/envs/ml/lib/python3.9/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


### Training the resnet model

In [12]:
history = dnn_model.fit(
train_set,
validation_data=valid_set,
epochs=5
)

Epoch 1/5


10/10 [==============================] - 11s 996ms/step - loss: 1.8338 - accuracy: 0.4437 - val_loss: 1.1433 - val_accuracy: 0.5625
Epoch 2/5
10/10 [==============================] - 10s 976ms/step - loss: 0.6189 - accuracy: 0.7750 - val_loss: 0.7111 - val_accuracy: 0.7344
Epoch 3/5
10/10 [==============================] - 10s 979ms/step - loss: 0.4160 - accuracy: 0.8531 - val_loss: 0.5352 - val_accuracy: 0.8094
Epoch 4/5
10/10 [==============================] - 10s 989ms/step - loss: 0.1918 - accuracy: 0.9438 - val_loss: 0.5052 - val_accuracy: 0.8062
Epoch 5/5
10/10 [==============================] - 10s 1s/step - loss: 0.1854 - accuracy: 0.9438 - val_loss: 0.5094 - val_accuracy: 0.8250


In [20]:
dnn_model.save_weights('resnet_no_augmentation_full_dataset_epochs10.h5')


# VGG Model

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers.legacy import Adam

vgg_model = Sequential()

imported_model = tf.keras.applications.VGG19(
    include_top=False,  # Do not include the top (fully connected) layers
    input_shape=(180, 180, 3),  # VGG19 default input size is 224x224
    pooling='avg',  # Add global average pooling
    weights='imagenet'  # Use weights pre-trained on ImageNet
)

for layer in imported_model.layers:
    layer.trainable = False

vgg_model.add(imported_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(256, activation='relu'))
vgg_model.add(Dense(10, activation='softmax'))  # Adjust the number of units according to your task

vgg_model.compile(
    optimizer=Adam(lr=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [17]:
history = vgg_model.fit(
train_set,
validation_data=valid_set,
epochs=5
)

Epoch 1/5
10/10 [==============================] - 32s 3s/step - loss: 2.0243 - accuracy: 0.5688 - val_loss: 1.7145 - val_accuracy: 0.5781
Epoch 2/5
10/10 [==============================] - 33s 3s/step - loss: 0.7088 - accuracy: 0.8062 - val_loss: 1.1874 - val_accuracy: 0.7250
Epoch 3/5
10/10 [==============================] - 32s 3s/step - loss: 0.3845 - accuracy: 0.8813 - val_loss: 0.9531 - val_accuracy: 0.7531
Epoch 4/5
10/10 [==============================] - 33s 3s/step - loss: 0.3021 - accuracy: 0.9125 - val_loss: 1.0913 - val_accuracy: 0.7188
Epoch 5/5
10/10 [==============================] - 32s 3s/step - loss: 0.1389 - accuracy: 0.9531 - val_loss: 0.9782 - val_accuracy: 0.7656


## EfficientNetV2S

In [21]:
def preprocess_image(image, label):
    resized_image = tf.image.resize(image, [384, 384])  # Resize to the expected input size
    return resized_image, label

# Assuming `train_dataset` and `valid_dataset` are your original datasets
train_dataset_resized = train_set.map(preprocess_image)
valid_dataset_resized = valid_set.map(preprocess_image)

In [28]:
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam

# Load the EfficientNetV2S model with a custom input size
efficient_net = tf.keras.applications.EfficientNetV2S(
    include_top=True,  # Include the fully connected layer at the top
    weights="imagenet",  # Use weights pre-trained on ImageNet
    input_tensor=None,  # Optional custom input tensor
    input_shape=None,  # Automatically adapts to the input shape of the weights
    pooling=None,  # No additional pooling layer at the top
    classes=1000,  # Number of classes for the ImageNet dataset
    classifier_activation="softmax",  # Use softmax activation for the final layer
    include_preprocessing=True  # Include preprocessing according to the model's requirements
)

efficient_net.compile(
    optimizer=Adam(lr=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)




/Users/EdenChung/anaconda3/envs/ml/lib/python3.9/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [29]:
history = efficient_net.fit(
train_dataset_resized,
validation_data=valid_dataset_resized,
epochs=5
)

Epoch 1/5


10/10 [==============================] - 102s 10s/step - loss: 2.8147 - accuracy: 0.5656 - val_loss: 1.5411 - val_accuracy: 0.7531
Epoch 2/5
10/10 [==============================] - 95s 10s/step - loss: 0.5462 - accuracy: 0.8594 - val_loss: 0.8403 - val_accuracy: 0.8313
Epoch 3/5
10/10 [==============================] - 109s 11s/step - loss: 0.5167 - accuracy: 0.8813 - val_loss: 0.4979 - val_accuracy: 0.8594
Epoch 4/5
10/10 [==============================] - 108s 11s/step - loss: 0.3889 - accuracy: 0.8938 - val_loss: 0.8952 - val_accuracy: 0.8375
Epoch 5/5
10/10 [==============================] - 109s 11s/step - loss: 0.5691 - accuracy: 0.8813 - val_loss: 0.5980 - val_accuracy: 0.8438
